In [99]:
# Import
import requests
import pandas as pd
import numpy as np

import re

import urllib
import bs4
from urllib import request

import sys
!{sys.executable} -m pip install geopy
import geopy
from geopy.geocoders import Nominatim
from geopy import distance

from datetime import date
from datetime import timedelta

# ML
import seaborn as sns
import matplotlib.pyplot as plt
import base64
import string
import re
!pip install nltk
import nltk

from collections import Counter
from nltk.corpus import stopwords
from time import time
#from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
!pip install pywaffle
from pywaffle import Waffle

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

!pip install spacy
!pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.1.0/fr_core_news_sm-3.1.0.tar.gz

import spacy

nlp = spacy.load("fr_core_news_sm")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     |████████████████████████████████| 17.1 MB 166 kB/s            
  Preparing metadata (setup.py) ... done


/opt/conda/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'fr_core_news_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [100]:
# Adds the part about the user right here

class User:
    def __init__(self, address, date, price_max = 42):
        geolocator = Nominatim(user_agent="CulturalEvents")
        location = geolocator.geocode(address)
        self.latlon = (location.latitude, location.longitude)
        self.date = date
        
Naclanga = User("27 Boulevard Thomas Gobert, Palaiseau", date.today())

In [115]:
# Data Extraction

# Query to extract a specific frame of time. We have to change it for the user desired date
query = "date_start < " + str(Naclanga.date + timedelta(days=7)) + " AND date_start > "  + str(Naclanga.date)
url = "https://data.iledefrance.fr/api/records/1.0/search/?dataset=evenements-publics-cibul&q=&rows=100&q=(" + query + ")&sort=date_start"
req = requests.get(url)

df = pd.json_normalize(req.json()['records'])

important_keys = ["fields.title", "fields.latlon", "fields.date_start", "fields.tags", "fields.pricing_info", "fields.description",'fields.link', 'fields.free_text', 'fields.address', 'fields.department','fields.placename', 'fields.region', 'fields.date_end', 'fields.lang', 'geometry.type', 'geometry.coordinates', 'fields.city_district']
df = df[important_keys]

dict_important_keys = {elt : elt.replace("fields.", "") for elt in important_keys}
df.rename(columns = dict_important_keys, inplace=True)

df_to_treat = df.copy()
df_to_treat

,title,latlon,date_start,tags,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district
0,22e Traversée de Paris en véhicules d'époque,"[48.839724, 2.435617]",2022-01-09,NaN,Entrée libre,"À l’occasion de ce gigantesque rassemblement, ...",http://openagenda.com/event/22e-traversee-de-p...,"Pour ce premier rendez-vous de l’année, les fi...",Vincennes,Paris,Parvis Sud du Château de Vincennes,Île-de-France,2022-01-09,fr,Point,"[2.435617, 48.839724]",Paris 12e Arrondissement
1,22e Traversée de Paris hivernale en véhicules ...,"[48.847451, 2.439671]",2022-01-09,NaN,Entrée libre,Nouvelle édition de la Traversée de Paris en A...,http://openagenda.com/event/22e-traversee-de-p...,"À l’occasion de ce gigantesque rassemblement, ...",Vincennes,Val-de-Marne,Parvis sud du château de Vincennes,Île-de-France,2022-01-09,fr,Point,"[2.439671, 48.847451]",NaN
2,L’enfant Océan / Cie AsaNisiMAsa / mise en sc...,"[48.609589, 2.312712]",2022-01-09,THÉÂTRE MARIONNETTES,"TARIF ORANGE (6 €, 12 ou 18 €) + PASS FAMILLE",THÉÂTRE MARIONNETTES,http://openagenda.com/event/lenfant-ocean-cie-...,_**durée 1h**_ \n_**EN FAMILLE DÈS 8 ANS**_\n...,Rue Henri Douard 91220 BRETIGNY SUR ORGE,Essonne,THEATRE BRETIGNY,Île-de-France,2022-01-09,fr,Point,"[2.312712, 48.609589]",NaN
3,Info pratique | Pharmacie de garde dimanche 9 ...,"[48.826498, 2.278734]",2022-01-09,Santé Garde Pharmacie,NaN,Retrouvez les pharmacies de garde du mois de j...,http://openagenda.com/event/info-pratique-or-p...,Toutes les pharmacies ouvertes en journée et l...,"11, rue du Général Leclerc 92130",Hauts-de-Seine,Pharmacie Radjibaly,Île-de-France,2022-01-09,fr,Point,"[2.278734, 48.826498]",NaN
4,LES VICE VERSA,"[48.671594, 2.53885]",2022-01-09,"HUMOUR,QUINCY-SOUS-SENART,ESSONNE,ILE DE FRANCE",Tarifs: de 10€ à 25€,UN SHOW VISUEL ET HILARANT D’UN GENRE NOUVEAU ...,http://openagenda.com/event/les-vice-versa,Deux garçons de café Parisien rêvent et imagin...,Rue de Combs-la-Ville 91480 Quincy sous Sénart,Essonne,Salle Mère Marie Pia,Île-de-France,2022-01-09,fr,Point,"[2.53885, 48.671594]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Atelier de Design Thinking,"[48.712203, 2.201042]",2022-01-06,"design,atelier,design thinking",Gratuit sur inscription / Pass sanitaire non r...,Découvrez la célèbre méthode d’innovation coll...,http://openagenda.com/event/atelier-de-design-...,[Consultez le programme complet et inscrivez-v...,8 avenue de la Vauve - Palaiseau,Essonne,Université Paris-Saclay - The Design Spot,Île-de-France,2022-04-07,fr,Point,"[2.201042, 48.712203]",Lozère
96,WEST SIDE STORY (VOST),"[48.787607, 2.232427]",2022-01-06,NaN,NaN,DRAME MUSICAL DE STEVEN SPIELBERG AVEC ANSEL E...,http://openagenda.com/event/west-side-story-vo...,L'histoire légendaire d'un amour naissant sur ...,"16 avenue du Maréchal de Lattre de Tassigny, 9...",Hauts-de-Seine,Espace culturel Robert-Doisneau,Île-de-France,2022-01-11,fr,Point,"[2.232427, 48.787607]",Meudon-la-Forêt
97,« Classique &amp;amp; mix » : une grande œuvre...,"[48.852385, 2.279006]",2022-01-06,NaN,NaN,L’Orchestre Philharmonique de Radio France pré...,http://openagenda.com/event/classique-andamp-m...,"Le 6 janvier 2022 à 20h, l’Orchestre Philharmo...","116 Avenue du Président Kennedy, 75016, Paris",NaN,Maison de la Radio et de la Musique,NaN,2022-01-06,fr,Point,"[2.279006, 48.852385]",NaN
98,FORMATION OPERATEUR LOGISITQUE,"[48.644306, 2.753786]",2022-01-06,"AFTRAL,FORMATION,OPERATEUR LOGISTIQUE",*sur inscription,LE CENTRE AFTRAL DE GARONOR-EST forme,http://openagenda.com/event/formation-operateu...,FORMATION OPERATEUR LOGISITQUE\n--------------...,Île-de-France,Seine-et-Marne,---,Île-de-France,2022-01-27,fr,Point,"[2.753786, 48.644306]",NaN


In [187]:
def extract_price_regex(x):
    L = []
    for k in re.findall("\S+\s?€", str(x)):
        try:
            a = float(k.replace("€", "").replace(",", "."))
        except ValueError:
            # It doesn't work
            a = float("NaN")
        L.append(a)
    return np.mean(L)

df_to_treat['pricing_info'] = df['pricing_info'].apply(lambda x:extract_price_regex(x))

In [188]:
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,NaN,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),NaN,Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,NaN,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,15.0,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,NaN,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, rue des Barres, 75004 Paris",Paris,"Église Saint-Gervais,Paris",Île-de-France,2022-06-05,fr,Point,"[2.354704, 48.855518]",Paris 4e Arrondissement


In [43]:
df_to_treat['distance'] = df['latlon'].apply(lambda x: distance.distance(x, Naclanga.latlon))
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district,distance
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,Inscription,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry,19.36843905054484 km
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN,24.29276779242778 km
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),"sur réservations , tarifs 13 euros TP et 10 eu...",Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais,17.92231496774433 km
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,Demandez votre badge gratuit,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN,14.797863061932679 km
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement,14.809879786818652 km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,Tarif plein : 20€ / réduit : 15€ / super rédui...,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN,7.771512371269672 km
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,Entrée libre sans rendez-vous,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN,13.591714003213303 km
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement,18.670708459014364 km
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, ru

In [189]:
# Date : we extract the hours of opening

In [190]:
def get_hours_regex(text):
    return str(text.select('span.hour')[0]).replace('<span class="hour">', "").replace("</span>", "")
    
def get_text(link):
    request_text = request.urlopen(link).read()
    page = bs4.BeautifulSoup(request_text)
    return(page)
    
def get_timespan(df_link):
    L = []
    for link in df_link:
        try:
            text = get_text(link)
            hours = get_hours_regex(text)
            L.append(hours)
        except (urllib.error.HTTPError, IndexError): #403 error case, if the Event is not correctly setup and if the event is not accessible
            L.append(float("NaN"))
        
    return L

df_to_treat['opening_hours'] = get_timespan(df['link']) # Time consuming, so we should get all information at once to avoid doing too much webscraping

# Should we differency ponctual events (movies, plays, sports) and long running events (expositions) ?
# -> We can probably do it judging the timespan of the events in terms of days and hours, but is it useful ?

In [191]:
df_to_treat

,title,latlon,date_start,tags,title,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district,opening_hours
0,Colloque AVISA,"[48.631891, 2.431379]",2022-12-09,NaN,Colloque AVISA,NaN,Écrire l’histoire du harcèlement sexuel sur la...,http://openagenda.com/event/colloque-avisa,Colloque en hybride,2 rue Facteur Cheval - Evry-Courcouronnes,Essonne,Université d'Évry - UFR SHS,Île-de-France,2022-12-10,fr,Point,"[2.431379, 48.631891]",Évry,09:00 - 17:00
1,Le Carnaval des Animaux Sud-américains,"[48.889308, 2.393804]",2022-11-20,"Ensemble ALMAVIVA,Carnaval des animaux sud-amé...",Le Carnaval des Animaux Sud-américains,NaN,"Bienvenue à Rio de Janeiro, qui accueille le c...",http://openagenda.com/event/le-carnaval-des-an...,Doté depuis son enfance à Rio du pouvoir de co...,"221 avenue Jean-Jaurès, 75019 Paris",Paris,La Philharmonie de Paris,Île-de-France,2022-11-20,fr,Point,"[2.393804, 48.889308]",NaN,11:00 - 12:00
2,Logique des passions (titre provisoire),"[48.800061, 2.403721]",2022-11-17,"théâtre,immersion,installation,numérique,vidéo...",Logique des passions (titre provisoire),NaN,Le Diptyque Collectif,http://openagenda.com/event/logique-des-passio...,Ce spectacle invite le public à vivre ou reviv...,"13, rue Pierre Sémard. 94400 Vitry-sur-Seine",Val-de-Marne,Nouveau Gare au Théâtre,Île-de-France,2022-11-19,fr,Point,"[2.403721, 48.800061]",Port à l'Anglais,20:00 - 21:00
3,Expoprotection,"[48.832528, 2.287004]",2022-11-15,"expoprotection,santé et sécurité au travail,sé...",Expoprotection,NaN,Le salon de la prévention et de la gestion des...,http://openagenda.com/event/expoprotection_640,"Tous les deux ans à Paris, Expoprotection rass...","Place de la Porte de Versailles, 75015 Paris",Paris,Porte de Versailles,Île-de-France,2022-11-17,fr,Point,"[2.287004, 48.832528]",NaN,09:00 - 18:00
4,EquipHotel,"[48.832488, 2.287499]",2022-11-06,"hôtellerie,restauration,collectivités",EquipHotel,NaN,Hotel &amp; Restaurant Business Place,http://openagenda.com/event/equiphotel_359,"Du 6 au 10 novembre 2022, ne manquez pas le re...",1 place de la Porte de Versailles,Paris,Paris expo Porte de Versailles,Île-de-France,2022-11-10,fr,Point,"[2.287499, 48.832488]",15e Arrondissement,09:30 - 19:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Sirba Octet &amp; Balalaïka,"[48.765358, 2.268554]",2022-06-07,NaN,Sirba Octet &amp; Balalaïka,15.0,SIRBALALAÏKA,http://openagenda.com/event/test-sirba-octet-a...,"Voir un concert du Sirba Octet, c’est comme pa...","254, avenue de la Division Leclerc 92290 châte...",Hauts-de-Seine,L'Azimut - Théâtre La Piscine,Île-de-France,2022-06-07,fr,Point,"[2.268554, 48.765358]",NaN,20:30 - 22:00
96,L'Aparté vous accueille ce samedi,"[48.824529, 2.274148]",2022-06-04,"CLAVIM,Aparté,Maison Verte",L'Aparté vous accueille ce samedi,NaN,Lieu d’accueil pour les bébés et les enfants d...,http://openagenda.com/event/laparte-vous-accue...,L’**Aparté** vous accueille sans rendez-vous c...,"60 rue du Général Leclerc, 92130 Issy-les-Moul...",Hauts-de-Seine,L'Aparté,Île-de-France,2022-06-04,fr,Point,"[2.274148, 48.824529]",NaN,15:00 - 18:30
97,Erstkommunion,"[48.870856, 2.285033]",2022-06-04,NaN,Erstkommunion,NaN,Festgottesdienst der Erstkommunion in St. Hono...,http://openagenda.com/event/erstkommunion-4029827,NaN,"66 bis avenue Raymond-Poincaré, 75116 Paris",Paris,St Honoré d'Eylau,Île-de-France,2022-06-04,fr,Point,"[2.285033, 48.870856]",Paris 16e Arrondissement,10:30 - 11:50
98,Solennité de Pentecôte,"[48.855518, 2.354704]",2022-06-04,NaN,Solennité de Pentecôte,NaN,Célébrer le don de l'Esprit Saint avec les Fra...,http://openagenda.com/event/solennite-de-pente...,* **Samedi 4 juin 2022**\n\n12h30 : messe du...,"13, rue des Barres, 75004 Paris",Paris,"Église Saint-Gervais,Paris",Île-de-France,2022-06-05,fr,Point,"[2.354704, 48.855518]",Paris 4e Arrondissement,12:30 - 13:00


In [5]:
database = pd.read_csv('events_database.zip', compression='zip', header=0, sep=';')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
database[['Description', 'Détails', 'Mots clés']].head(5)

,Description,Détails,Mots clés
0,Découvrir le poney dans un séjour multisports ...,L'UCPA propose ses vacances apprenantes : Un s...,"UCPA,sport,nature,SFJBOI31E,import-ucpa-20200630"
1,"Un séjour qui combine sport, collectivité et f...",Cette colo te permettra de découvrir les multi...,"jeux olympiques,sport,import-pep"
2,Le top de la glisse de la côte californienne,L'UCPA propose ses vacances apprenantes : Un s...,"UCPA,sport,nature,SFJPON34R,import-ucpa-20200630"
3,Compagnie AtmosphèreS - mise en scène : Sara V...,"Camille, Camille, Camille\n-------------------...","Camille Claudel,Rodin,Paul claudel,sculptrice,..."
4,Mise en scène par Agnès Braunschweig. Cie Pros...,Je reviens de la vérité de Charlotte Delbo. Mi...,"Résistance,Déportation,Deuxième guerre mondial..."


In [107]:
def NaN_to_String(dataframe):
    df_temp = dataframe.copy()
    df_temp[df_temp.isna()] = ""
    return df_temp.apply(lambda s: s.replace("\n", "").replace("\r", "") + ". ")

text_desc = (NaN_to_String(database['Description']) + NaN_to_String(database['Détails']) + NaN_to_String(database['Mots clés']).apply(lambda s: s.replace(",", ", ")))

In [21]:
text = text_desc.head(1000).sum().lower()
words = nltk.word_tokenize(text, language='french')
words = [word for word in words if word.isalpha()]
words = [w for w in words if not w in stop_words]
dic_words = Counter(words)

In [22]:
dic_words.most_common(50)

[('a', 380),
 ('plus', 268),
 ('paris', 256),
 ('cette', 220),
 ('musique', 203),
 ('http', 167),
 ('amp', 157),
 ('tout', 156),
 ('entre', 153),
 ('théâtre', 145),
 ('https', 133),
 ('monde', 130),
 ('tous', 127),
 ('comme', 121),
 ('concert', 117),
 ('scène', 116),
 ('leurs', 113),
 ('visite', 113),
 ('découvrir', 112),
 ('samedi', 112),
 ('rock', 108),
 ('france', 105),
 ('aussi', 101),
 ('the', 100),
 ('atelier', 100),
 ('public', 99),
 ('deux', 96),
 ('musée', 96),
 ('faire', 94),
 ('projet', 94),
 ('dimanche', 94),
 ('spectacle', 93),
 ('autour', 89),
 ('où', 89),
 ('fait', 89),
 ('groupe', 88),
 ('propose', 87),
 ('création', 87),
 ('ans', 85),
 ('jazz', 85),
 ('enfants', 84),
 ('place', 83),
 ('sous', 83),
 ('jeunes', 82),
 ('travers', 81),
 ('depuis', 80),
 ('cadre', 79),
 ('ateliers', 79),
 ('art', 79),
 ('vie', 77)]

In [224]:
def return_POS(sentence, n = -1):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner les étiquettes de chaque token
    words = [[X.lemma_, X.lemma_] for X in doc if (X.is_alpha == True and X.pos_ == "NOUN" and not(X in stop_words))]
    df_words = pd.DataFrame(words, columns=['tags', 'unique_tags'])
    if n == -1:
        return list(df_words.groupby(['unique_tags']).count().sort_values('tags', ascending=False).to_dict()['tags'].keys())
    else:
        return list(df_words.groupby(['unique_tags']).count().sort_values('tags', ascending=False).head(n).to_dict()['tags'].keys())

In [225]:
tags = return_POS(text_desc.head(1000).sum().lower(), 50)
tags

['musique',
 'atelier',
 'théâtre',
 'concert',
 'projet',
 'visite',
 'scène',
 'monde',
 'histoire',
 'année',
 'groupe',
 'samedi',
 'exposition',
 'jeu',
 'public',
 'enfant',
 'spectacle',
 'an',
 'place',
 'dimanche',
 'création',
 'rencontre',
 'conférence',
 'recherche',
 'association',
 'vie',
 'aide',
 'travers',
 'cadre',
 'découverte',
 'festival',
 'temps',
 'artiste',
 'maison',
 'formation',
 'œuvre',
 'danse',
 'siècle',
 'musée',
 'travail',
 'ville',
 'soirée',
 'journée',
 'heure',
 'université',
 'occasion',
 'programme',
 'chanson',
 'centre',
 'rue']

In [226]:
df_to_treat['text'] = (NaN_to_String(df['tags']) + NaN_to_String(df['title']) + NaN_to_String(df['description']) + NaN_to_String(df['free_text'])).apply(lambda s: s.replace(",", ", "))
df_to_treat['tags'] = df_to_treat['text'].apply(lambda x:return_POS(x.lower()))
df_to_treat['tags'] = df_to_treat['tags'].apply(lambda x: [k for k in x if k in tags])

In [227]:
df_to_treat['tags'] 

0     [place, spectacle, année, histoire, musée, an,...
1     [place, année, spectacle, histoire, musée, an,...
2     [enfant, théâtre, scène, spectacle, histoire, ...
3                                   [dimanche, journée]
4                                    [spectacle, danse]
                            ...                        
95    [atelier, année, occasion, programme, université]
96                                           [histoire]
97    [œuvre, artiste, musique, scène, exposition, s...
98                           [formation, centre, place]
99                      [spectacle, musique, an, scène]
Name: tags, Length: 100, dtype: object